# About

- Workbook is implemented to better understand the recommender system.
- Data is downloaded from Kaggle datasets and full version is available [here](https://www.kaggle.com/rounakbanik/the-movies-dataset/data).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_data = pd.read_csv('imdb_ratings_small.csv', dtype=str, usecols=['userId', 'movieId', 'rating'])
df_imdb = pd.read_csv('imdb_links_small.csv', dtype=str, usecols=['movieId', 'imdbId'])
movieId_to_imbdId = {k: v for k, v in zip(df_imdb.movieId.tolist(), df_imdb.imdbId.tolist())}
df_data.movieId = df_data.movieId.apply(lambda x: movieId_to_imbdId[x])
df_data.rating = df_data.rating.apply(float)

In [ ]:
from keras.preprocessing.text import Tokenizer
movie_tokenizer = Tokenizer()
movie_tokenizer.fit_on_texts(df_data.movieId.tolist())
df_data.movieId = [_[0] for _ in movie_tokenizer.texts_to_sequences(df_data.movieId.tolist())]

user_tokenizer = Tokenizer()
user_tokenizer.fit_on_texts(df_data.userId.tolist())
df_data.userId = [_[0] for _ in user_tokenizer.texts_to_sequences(df_data.userId.tolist())]

In [ ]:
y = np.zeros((
    df_data.movieId.max(axis=0)+1, #movies+1
    df_data.userId.max(axis=0)+1,  #users+1
))
print("initialized y with size:", y.shape)

for idx in df_data.userId.unique().tolist():
    y[df_data[df_data.userId==idx].movieId.tolist(), idx] = df_data[df_data.userId==idx].rating.tolist()

In [ ]:
y = np.array(
    [
        [3. , 0. , 4.5, 4. , 2. ],
        [3. , 4. , 3.5, 5. , 3. ],
        [0. , 0. , 3. , 5. , 3. ],
        [4. , 0. , 3. , 0. , 0. ],
        [0. , 0. , 5. , 5. , 3.5],
        [0. , 0. , 5. , 4. , 3.5],
        [0. , 5. , 5. , 5. , 4.5],
        [4. , 4. , 2.5, 5. , 0. ],
        [0.5, 0. , 4. , 0. , 2.5],
        [0. , 0. , 0. , 4. , 0. ]
    ]
)
r = np.where(y > 0, 1, 0)

In [ ]:
y_small = y[100:110, 50:55]

In [ ]:
print("Shape:")
print("\ty:", y_small.shape)

In [ ]:
def estimate_x(y, max_k=2, x=None, theta=None,
               _alpha = 0.01, _lambda=0.001, _tolerance = 0.001):
    r = np.where(y > 0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    if type(x) != np.array:
        x = np.random.randn(max_i, max_k)
    if type(theta) != np.array:
        theta = np.random.randn(max_k+1, max_j)
    _x = x.copy()
    while not converged:
        for i in range(max_i):
            iter_y = y[i][np.where(r[i] != 0)]
            iter_theta = theta.transpose()[np.where(r[i] != 0)].transpose()
            iter_x_with_bias = np.hstack((np.ones((1,)), x[i]))
            y_hat = np.matmul(iter_x_with_bias, iter_theta)
            diff = y_hat - iter_y
            del_x = np.matmul(iter_theta, diff)
            update_x = _alpha * (del_x[1:]  + _lambda * x[i])
            x[i] = x[i] - update_x
        if np.max(abs(update_x)) < _tolerance:
            converged = True
    return theta, x

def estimate_theta(y, max_k=2, x=None, theta=None,
               _alpha = 0.01, _lambda=0.001, _tolerance = 0.001):
    r = np.where(y > 0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    if type(x) != np.array:
        x = np.random.randn(max_i, max_k)
    if type(theta) != np.array:
        theta = np.random.randn(max_j, max_k+1)
    _theta = theta.copy()
    while not converged:
        for j in range(max_j):
            included_rows = np.where(r[:, j] != 0)
            iter_y = y[:, j][included_rows]
            iter_theta = theta[:, j]
            iter_x = x[included_rows]
            iter_x_with_bias = np.hstack((np.ones((iter_x.shape[0],1)), iter_x))
            y_hat = np.matmul(iter_x_with_bias, iter_theta)
            diff = y_hat - iter_y
            del_theta = np.matmul(iter_x_with_bias.transpose(), diff)
            update_theta = del_theta
            update_theta[1:] += _lambda * iter_theta[1:]
            update_theta *= _alpha
            theta[:, j] = theta[:, j] - update_theta
        if np.max(abs(update_theta)) < _tolerance:
            converged = True
    return theta, x

In [ ]:
def estimate_x_v2(y, max_k=2, x=None, theta=None,
               _alpha = 0.01, _lambda=0.001, _tolerance = 0.001):
    r = np.where(y > 0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    if type(x) != np.array:
        x = np.random.randn(max_i, max_k)
    if type(theta) != np.array:
        theta = np.random.randn(max_j, max_k+1)
    while not converged:
        update_x = np.zeros(x.shape)
        update_x = _alpha * (
            np.matmul(
                (
                    np.matmul(
                        np.hstack((np.ones((x.shape[0], 1)),x)), 
                        theta.transpose()
                    ) - y
                ) * r, 
                theta
            )[:, 1:] + _lambda * x
        )
        x = x - update_x
        if np.max(abs(update_x)) < _tolerance:
            converged = True
    return theta, x

def estimate_theta_v2(y, max_k=2, x=None, theta=None,
               _alpha = 0.01, _lambda=0.001, _tolerance = 0.001):
    r = np.where(y > 0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    if type(x) != np.array:
        x = np.random.randn(max_i, max_k)
    if type(theta) != np.array:
        theta = np.random.randn(max_j, max_k+1)
    while not converged:
        update_theta = np.zeros(theta.shape)
        update_theta = _alpha * (
            np.matmul(
                np.hstack((np.ones((x.shape[0], 1)),x)).transpose(),
                (
                    np.matmul(
                        np.hstack((np.ones((x.shape[0], 1)),x)), 
                        theta.transpose()
                    ) - y
                ) * r, 
            ).transpose() + _lambda * theta
        )
        theta = theta - update_theta
        if np.max(abs(update_theta)) < _tolerance:
            converged = True
    return theta, x

In [ ]:
def colaborative_filtering(y, max_k=2,
                         _alpha=0.01, _lambda=0.001, _tolerance=0.001):
    r = np.where(y>0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    x = np.random.rand(max_i, max_k)
    theta = np.random.rand(max_k, max_j)
    _x = x.copy()
    _theta = theta.copy()
    while not converged:
        update_x = np.zeros(x.shape)
        update_theta = np.zeros(theta.shape)
        for i in range(max_i):
            iter_y = y[i][np.where(r[i] != 0)]
            iter_theta = theta.transpose()[np.where(r[i] != 0)].transpose()
            iter_x = x[i]
            y_hat = np.matmul(iter_x, iter_theta)
            diff = y_hat - iter_y
            del_x = np.matmul(iter_theta, diff)
            update_x[i] = _alpha * (del_x  + _lambda * x[i])
        for j in range(max_j):
            included_rows = np.where(r[:, j] != 0)
            iter_y = y[:, j][included_rows]
            iter_theta = theta[:, j]
            iter_x = x[included_rows]
            y_hat = np.matmul(iter_x, iter_theta)
            diff = y_hat - iter_y
            del_theta = np.matmul(iter_x.transpose(), diff)
            update_theta[:, j] = _alpha * (del_theta + _lambda * iter_theta)
        x = x - update_x    
        theta = theta - update_theta
        if max(np.max(abs(update_x)), np.max(abs(update_theta))) < _tolerance:
            converged = True
    return theta, x

In [ ]:
def colaborative_filtering_v2(y, max_k=2,
             _alpha=0.01, _lambda=0.001, _tolerance=0.001, r=None):
    if type(r) != np.ndarray:
        r = np.where(y>0, 1, 0)
    converged = False
    max_i, max_j = y.shape
    x = np.random.rand(max_i, max_k)
    theta = np.random.rand(max_j, max_k)
    
    while not converged:
        update_x = np.zeros(x.shape)
        update_theta = np.zeros(theta.shape)
        update_x = _alpha * (
            np.matmul(
                (np.matmul(x, theta.transpose()) - y) * r, 
                theta
            ) + _lambda * x
        )
        update_theta = _alpha * (
            np.matmul(
                x.transpose(),
                (np.matmul(x, theta.transpose()) - y) * r, 
            ).transpose() + _lambda * theta
        )
        x = x - update_x
        theta = theta - update_theta
        if max(np.max(abs(update_x)), np.max(abs(update_theta))) < _tolerance:
            converged = True
    return theta, x

In [ ]:
tolerance=0.001
max_k=50

In [ ]:
theta, x = estimate_x_v2(y, _tolerance=tolerance, max_k=max_k)

In [ ]:
for _ in range(2):
    theta, x = estimate_theta_v2(y, x=x, theta=theta, _tolerance=tolerance, max_k=max_k)
    theta, x = estimate_x_v2(y, x=x, theta=theta, _tolerance=tolerance, max_k=max_k)

In [ ]:
y

In [ ]:
np.matmul(np.hstack((np.ones((10, 1)), x)), theta.transpose()).round(decimals=2)

In [ ]:
theta, x = colaborative_filtering_v2(y, max_k=max_k)

In [ ]:
y

In [ ]:
np.matmul(x, theta.transpose()).round(decimals=2)

In [ ]:
y = np.hstack((y, np.zeros((y.shape[0], 1))))

In [ ]:
max_k = 5
tolerance = 0.0000001
theta, x = colaborative_filtering_v2(y, max_k=max_k, _tolerance=tolerance)

In [ ]:
y

In [ ]:
np.matmul(x, theta.transpose()).round(decimals=2)

In [ ]:
def normalized(y, max_k=2,
             _alpha=0.01, _lambda=0.001, _tolerance=0.001):
    r = np.where(y>0, 1, 0)
    y_sum = y.sum(axis=1)
    r_sum = r.sum(axis=1)
    y_mean = np.atleast_2d(y_sum/r_sum).transpose()
    y_norm = y - y_mean
    theta, x = colaborative_filtering_v2(y_norm, max_k, _alpha, _lambda, _tolerance, r)
    return theta, x, y_mean

In [ ]:
theta, x, y_mean = normalized(y, max_k=max_k, _tolerance=tolerance)

In [ ]:
y

In [ ]:
(np.matmul(x, theta.transpose()) + y_mean).round(decimals=2)